In [1]:
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools

In [2]:
class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vector({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))

    def __hash__(self):
        hashes = (hash(x) for x in self)
        return functools.reduce(operator.xor, hashes, 0)

# BEGIN VECTOR_V6_UNARY
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))

    def __neg__(self):
        return Vector(-x for x in self)  # <1>

    def __pos__(self):
        return Vector(self)  # <2>
# END VECTOR_V6_UNARY

    def __bool__(self):
        return bool(abs(self))

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers'
            raise TypeError(msg.format(cls))

    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))

    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a

    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):  # hyperspherical coordinates
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)],
                                     self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

# BEGIN VECTOR_V6_ADD
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented

    def __radd__(self, other):
        return self + other

#### A multi-dimensional ``Vector`` class, take 6: operator ``+``
A ``Vector`` is built from an iterable of numbers::

In [3]:
>>> Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [4]:
>>> Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [5]:
>>> Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

#### Tests with 2-dimensions (same results as ``vector2d_v1.py``):

In [6]:
>>> v1 = Vector([3, 4])
>>> x, y = v1
>>> x, y

(3.0, 4.0)

In [7]:
>>> v1

Vector([3.0, 4.0])

In [8]:
>>> v1_clone = eval(repr(v1))
>>> v1 == v1_clone

True

In [9]:
>>> print(v1)

(3.0, 4.0)


In [10]:
>>> octets = bytes(v1)
>>> octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [11]:
>>> abs(v1)

5.0

In [12]:
>>> bool(v1), bool(Vector([0, 0]))

(True, False)

#### Test of ``.frombytes()`` class method:

In [13]:
>>> v1_clone = Vector.frombytes(bytes(v1))
>>> v1_clone

Vector([3.0, 4.0])

In [14]:
>>> v1 == v1_clone

True

#### Tests with 3-dimensions::

In [15]:
>>> v1 = Vector([3, 4, 5])
>>> x, y, z = v1
>>> x, y, z

(3.0, 4.0, 5.0)

In [16]:
>>> v1

Vector([3.0, 4.0, 5.0])

In [17]:
>>> v1_clone = eval(repr(v1))
>>> v1 == v1_clone

True

In [18]:
>>> print(v1)

(3.0, 4.0, 5.0)


In [19]:
>>> abs(v1)  # doctest:+ELLIPSIS

7.0710678118654755

In [20]:
>>> bool(v1), bool(Vector([0, 0, 0]))

(True, False)

#### Tests with many dimensions::

In [21]:
>>> v7 = Vector(range(7))
>>> v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [22]:
>>> abs(v7)  # doctest:+ELLIPSIS

9.539392014169456

#### Test of ``.__bytes__`` and ``.frombytes()`` methods::

In [23]:
>>> v1 = Vector([3, 4, 5])
>>> v1_clone = Vector.frombytes(bytes(v1))
>>> v1_clone

Vector([3.0, 4.0, 5.0])

In [24]:
>>> v1 == v1_clone

True

#### Tests of sequence behavior::

In [25]:
>>> v1 = Vector([3, 4, 5])
>>> len(v1)

3

In [26]:
>>> v1[0], v1[len(v1)-1], v1[-1]

(3.0, 5.0, 5.0)

#### Test of slicing::

In [27]:
>>> v7 = Vector(range(7))
>>> v7[-1]

6.0

In [28]:
>>> v7[1:4]

Vector([1.0, 2.0, 3.0])

In [29]:
>>> v7[-1:]

Vector([6.0])

In [30]:
>>> v7[1,2]

TypeError: Vector indices must be integers

#### Tests of dynamic attribute access::

In [31]:
>>> v7 = Vector(range(10))
>>> v7.x

0.0

In [32]:
>>> v7.y, v7.z, v7.t

(1.0, 2.0, 3.0)

#### Dynamic attribute lookup failures::

In [33]:
>>> v7.k

AttributeError: 'Vector' object has no attribute 'k'

In [34]:
>>> v3 = Vector(range(3))
>>> v3.t

AttributeError: 'Vector' object has no attribute 't'

In [35]:
>>> v3.spam

AttributeError: 'Vector' object has no attribute 'spam'

#### Tests of hashing::

In [36]:
>>> v1 = Vector([3, 4])
>>> v2 = Vector([3.1, 4.2])
>>> v3 = Vector([3, 4, 5])
>>> v6 = Vector(range(6))
>>> hash(v1), hash(v3), hash(v6)

(7, 2, 1)

#### Most hash values of non-integers vary from a 32-bit to 64-bit Python build::

In [37]:
>>> import sys
>>> hash(v2) == (384307168202284039 if sys.maxsize > 2**32 else 357915986)

True

#### Tests of ``format()`` with Cartesian coordinates in 2D::

In [38]:
>>> v3 = Vector([3, 4, 5])
>>> format(v3)

'(3.0, 4.0, 5.0)'

In [39]:
>>> format(Vector(range(7)))

'(0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0)'

#### Tests of ``format()`` with spherical coordinates in 2D, 3D and 4D::

In [40]:
>>> format(Vector([1, 1]), 'h')  # doctest:+ELLIPSIS

'<1.4142135623730951, 0.7853981633974483>'

In [41]:
>>> format(Vector([1, 1]), '.3eh')

'<1.414e+00, 7.854e-01>'

In [42]:
>>> format(Vector([1, 1]), '0.5fh')

'<1.41421, 0.78540>'

In [43]:
>>> format(Vector([1, 1, 1]), 'h')  # doctest:+ELLIPSIS

'<1.7320508075688772, 0.9553166181245093, 0.7853981633974483>'

In [44]:
>>> format(Vector([2, 2, 2]), '.3eh')

'<3.464e+00, 9.553e-01, 7.854e-01>'

In [45]:
>>> format(Vector([0, 0, 0]), '0.5fh')

'<0.00000, 0.00000, 0.00000>'

In [46]:
>>> format(Vector([-1, -1, -1, -1]), 'h')  # doctest:+ELLIPSIS

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [47]:
>>> format(Vector([2, 2, 2, 2]), '.3eh')

'<4.000e+00, 1.047e+00, 9.553e-01, 7.854e-01>'

In [48]:
>>> format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'

#### Unary operator tests::

In [49]:
>>> v1 = Vector([3, 4])
>>> abs(v1)

5.0

In [50]:
>>> -v1

Vector([-3.0, -4.0])

In [51]:
>>> +v1

Vector([3.0, 4.0])

#### Basic tests of operator ``+``::

In [52]:
>>> v1 = Vector([3, 4, 5])
>>> v2 = Vector([6, 7, 8])
>>> v1 + v2

Vector([9.0, 11.0, 13.0])

In [53]:
>>> v1 + v2 == Vector([3+6, 4+7, 5+8])

True

In [54]:
>>> v3 = Vector([1, 2])
>>> v1 + v3  # short vectors are filled with 0.0 on addition

Vector([4.0, 6.0, 5.0])

#### Tests of ``+`` with mixed types::

In [55]:
>>> v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [57]:
import import_ipynb
from vector2d_v3 import Vector2d
v2d = Vector2d(1, 2)
v1 + v2d

Vector([4.0, 6.0, 5.0])

#### Tests of ``+`` with mixed types, swapped operands::

In [58]:
>>> (10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

In [59]:
>>> from vector2d_v3 import Vector2d
>>> v2d = Vector2d(1, 2)
>>> v2d + v1

Vector([4.0, 6.0, 5.0])

#### Tests of ``+`` with an unsuitable operand:

In [60]:
>>> v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [61]:
>>> v1 + 'ABC'

TypeError: unsupported operand type(s) for +: 'Vector' and 'str'